### Things to calculate:
total/mt+/mt-
 - Pi N
 - Pi S
 - Theta W
 - S
 - TajD

between
 - Fst between alleles
 - fixed/shared/private alleles
total
 - GC content at synonymous sites
 
Codon stats


In [21]:
from ness_fasta import ness_fasta
from Bio import SeqIO, AlignIO
from collections import Counter
from ness_vcf import SFS
import glob
import pandas as pd

In [22]:
def get_column(position, aln, samples=None):
    #sample_IDs = [sample.id.split("|")[0] for sample in aln]
    sample_IDs = [s.id for s in aln]
    column = aln[:,position]
    col = dict(zip(sample_IDs, [i for i in column]))
    if samples != None:
        col_filtered ={}
        for s in col:
            if s in samples:
                col_filtered[s]=col[s]
        col=col_filtered
    return col

def alignment2sfs(aln, min_called, samples=None, start=None, end=None):
    if start== None: start =0
    if end == None:end = len(aln[0].seq)
    if samples == None:
        samples=[s.id for s in aln]
    sfs = SFS([0]*(len(samples)+1))
    for position in range(start, end):
        column = get_column(position, aln, samples)
        bases=column.values()
        callable_bases = [column[sampleID] for sampleID in samples if column[sampleID] in 'AaCcGgTt']
        if len(callable_bases) >=min_called: # if callable
            base_counts = Counter(callable_bases)
            if len(base_counts) == 1: # invariant
                sfs.add(0,len(samples))
            elif len(base_counts) >2:  # triallelic
                continue
            elif len(base_counts)==2:
                #print(len(base_counts), callable_bases)
                AF = Counter(bases)[list(Counter(bases).keys())[1]]/len(bases)
                sfs.add(AF,len(samples))
    return sfs

def concatenate_SFSs(SFSs):
    concatenate_sfs =[]
    for s in SFSs:
        for i in range(len(s)):
            try:combined_sfs[i] += s[i]
            except IndexError:
                "You can't combine SFSs with different numbers of samples"
                return None
    return combined_sfs

gen_code_dict = { 'ttt': 'F', 'ttc': 'F', 'tta': 'L', 'ttg': 'L', \
                'tct': 'S', 'tcc': 'S', 'tca': 'S', 'tcg': 'S', \
                'tat': 'Y', 'tac': 'Y', 'taa': 'X', 'tag': 'X', \
                'tgt': 'C', 'tgc': 'C', 'tga': 'X', 'tgg': 'W', \
                'ctt': 'L', 'ctc': 'L', 'cta': 'L', 'ctg': 'L', \
                'cct': 'P', 'ccc': 'P', 'cca': 'P', 'ccg': 'P', \
                'cat': 'H', 'cac': 'H', 'caa': 'Q', 'cag': 'Q', \
                'cgt': 'R', 'cgc': 'R', 'cga': 'R', 'cgg': 'R', \
                'att': 'I', 'atc': 'I', 'ata': 'I', 'atg': 'M', \
                'act': 'T', 'acc': 'T', 'aca': 'T', 'acg': 'T', \
                'aat': 'N', 'aac': 'N', 'aaa': 'K', 'aag': 'K', \
                'agt': 'S', 'agc': 'S', 'aga': 'R', 'agg': 'R', \
                'gtt': 'V', 'gtc': 'V', 'gta': 'V', 'gtg': 'V', \
                'gct': 'A', 'gcc': 'A', 'gca': 'A', 'gcg': 'A', \
                'gat': 'D', 'gac': 'D', 'gaa': 'E', 'gag': 'E', \
                'ggt': 'G', 'ggc': 'G', 'gga': 'G', 'ggg': 'G'}

degen_dict = {'ttt': '002', 'ttc': '002', 'tta': '202', 'ttg': '202', \
              'tct': '004', 'tcc': '004', 'tca': '004', 'tcg': '004', \
              'tat': '002', 'tac': '002', 'taa': '022', 'tag': '002', \
              'tgt': '002', 'tgc': '002', 'tga': '020', 'tgg': '000', \
              'ctt': '004', 'ctc': '004', 'cta': '204', 'ctg': '204', \
              'cct': '004', 'ccc': '004', 'cca': '004', 'ccg': '004', \
              'cat': '002', 'cac': '002', 'caa': '002', 'cag': '002', \
              'cgt': '004', 'cgc': '004', 'cga': '204', 'cgg': '204', \
              'att': '003', 'atc': '003', 'ata': '003', 'atg': '000', \
              'act': '004', 'acc': '004', 'aca': '004', 'acg': '004', \
              'aat': '002', 'aac': '002', 'aaa': '002', 'aag': '002', \
              'agt': '002', 'agc': '002', 'aga': '202', 'agg': '202', \
              'gtt': '004', 'gtc': '004', 'gta': '004', 'gtg': '004', \
              'gct': '004', 'gcc': '004', 'gca': '004', 'gcg': '004', \
              'gat': '002', 'gac': '002', 'gaa': '002', 'gag': '002', \
              'ggt': '004', 'ggc': '004', 'gga': '004', 'ggg': '004'}


def special_consensus(aln):
    consensus = ''
    for p in range(len(aln[0].seq)):
        called_bases = [b for b in aln[:,p] if b in "ACTG"]
        try:
            most_common_nucleotide = list(Counter(called_bases).keys())[0]
            consensus+=most_common_nucleotide
        except IndexError:
            consensus+='N'
    return consensus
        
def degenerate(seq):
    degen_seq = ''
    for idx in range(0, len(aln[0])-2,3):    
        codon = seq[idx:idx+3]
        try:
            degen  = degen_dict[codon.lower()]
        except KeyError:
            degen = 'nnn'
        degen_seq = degen_seq+degen
    return degen_seq

def alignment2sfs_w_degen(aln, min_called, degen_map, samples=None, start=None, end=None):
    if start== None: start =0
    if end == None:end = len(aln[0].seq)
    if samples == None:
        samples=[s.id for s in aln]
    sfs0 = SFS([0]*(len(samples)+1))
    sfs4 = SFS([0]*(len(samples)+1))
    sfstotal = SFS([0]*(len(samples)+1))
    for position in range(start, end):
        degeneracy = degen_map[position]
        column = get_column(position, aln, samples)
        bases=column.values()
        callable_bases = [column[sampleID] for sampleID in samples if column[sampleID] in 'AaCcGgTt']
        if len(callable_bases) >=min_called: # if callable
            base_counts = Counter(callable_bases)
            if len(base_counts) == 1: # invariant
                sfstotal.add(0,len(samples))
                if degeneracy=='0':sfs0.add(0,len(samples))
                if degeneracy=='4':sfs4.add(0,len(samples))
            elif len(base_counts) >2:  # triallelic
                continue
            elif len(base_counts)==2:
                #print(len(base_counts), callable_bases)
                AF = Counter(bases)[list(Counter(bases).keys())[1]]/len(bases)
                sfstotal.add(AF,len(samples))
                if degeneracy=='0':sfs0.add(AF,len(samples))
                if degeneracy=='4':sfs4.add(AF,len(samples))

    return [sfstotal, sfs0, sfs4]

def population_poly(aln, min_called, degen_map, group1, group2, start=None, end=None):
    ## Calculate stats between groups:
    #     SNP distribution stats:
    #     private = polymorphic in one group and not the other
    #     fixed = groups are monomorphic within but different
    #     shared = polymorphic in both
    ## Fst  - differentiation
    #     Fst = 1 - [ mean(2*p_1*q_1, 2*p_2*q_2) / (2*p_total*q_total) ]
    if start== None: start =0
    if end == None:end = len(aln[0].seq)
    group1_samples=[s.id for s in aln if s.id in group1]
    group2_samples=[s.id for s in aln if s.id in group2]
    all_samples=group1_samples+group2_samples
    fst = [0, 0]
    private1, private2,fixed,shared = 0,0,0,0 
    for position in range(start, end):
        degeneracy = degen_map[position]
        column = get_column(position, aln, all_samples)
        bases=column.values()
        all_callable_bases = [column[sampleID] for sampleID in all_samples if column[sampleID] in 'AaCcGgTt']
        group1_callable_bases = [column[sampleID] for sampleID in group1_samples if column[sampleID] in 'AaCcGgTt']
        group2_callable_bases = [column[sampleID] for sampleID in group2_samples if column[sampleID] in 'AaCcGgTt']
        if len(all_callable_bases) >=min_called: # if callable
            base_counts_all = Counter(all_callable_bases)
            base_counts_1 = Counter(group1_callable_bases)
            base_counts_2 = Counter(group2_callable_bases)
            #if this is monomorphic - move on!
            if len(base_counts_all) == 1:# invariant
                # Add invariant counts to all or do nothing  
                pass
            elif len(base_counts_all) >2:  # triallelic
                continue
            elif len(base_counts_all)==2:
                p_1 = base_counts_1[list(base_counts_1.keys())[0]]/len(group1_callable_bases)
                p_2 = base_counts_2[list(base_counts_2.keys())[0]]/len(group2_callable_bases)
                p_all = base_counts_all[list(base_counts_all.keys())[0]]/len(all_callable_bases)
                pq_1 = 2*p_1*(1-p_1)
                pq_2 = 2*p_2*(1-p_2)
                pq_within=(pq_1+pq_2)/2
                pq_all = 2*p_all*(1-p_all)
                fst_i = 1-(pq_within/pq_all)
                fst[0]+=fst_i #running sum of fst
                fst[1]+=1 # total sites in there
                if p_1 == 1.0 and 0<p_2<1:
                    private2+=1
                elif p_2 == 1.0 and 0<p_1<1:
                    private1+=1
#                     print(group1_callable_bases, group2_callable_bases, p_1, p_2)
                if 0<p_1<1 and 0<p_2<1:
                    shared+=1
                if p_1 ==1.0 and p_2 ==1.0 and 0<p_all<1:
                    fixed +=1                    
 #########################################################################################################
    try:fst =  fst[0]/fst[1]
    except ZeroDivisionError:
        fst = 'N/A'
    results = { 'private1': private1,
                'private2': private2,
                'fixed': fixed,
                'shared': shared,
                'fst' :fst}
    return results




In [23]:
# Target Genotype Lists
mtPlus_Genotypes = "CC2936 CC2937 CC3060 CC3064 CC3065 CC3068 CC3071 CC3076 CC3086".split()
mtMinus_Genotypes = "CC2935 CC2938 CC3059 CC3061 CC3062 CC3063 CC3073 CC3075 CC3079 CC3084".split()

In [34]:
#### STATS TO COLLECT for each gene
genes = []
ness_ids = []
mtPlus_coords = []
mtMinus_coords = []
alleles_ALL = []
sites_ALL = []
invariant_ALL = []
variant_ALL = []
theta_pi_0fold_ALL = []
theta_pi_4fold_ALL = []
theta_pi_total_ALL = []
theta_w_0fold_ALL = []
theta_w_4fold_ALL = []
theta_w_total_ALL = []
sfs_4fold_ALL = []
sfs_0fold_ALL = []
sfs_t_ALL = []
alleles_MTPLUS = []
sites_MTPLUS = []
invariant_MTPLUS = []
variant_MTPLUS = []
theta_pi_0fold_MTPLUS = []
theta_pi_4fold_MTPLUS = []
theta_pi_total_MTPLUS = []
theta_w_0fold_MTPLUS = []
theta_w_4fold_MTPLUS = []
theta_w_total_MTPLUS = []
sfs_4fold_MTPLUS = []
sfs_0fold_MTPLUS = []
sfs_t_MTPLUS = []
alleles_MTMINUS = []
sites_MTMINUS = []
invariant_MTMINUS = []
variant_MTMINUS = []
theta_pi_0fold_MTMINUS = []
theta_pi_4fold_MTMINUS = []
theta_pi_total_MTMINUS = []
theta_w_0fold_MTMINUS = []
theta_w_4fold_MTMINUS = []
theta_w_total_MTMINUS = []
sfs_4fold_MTMINUS = []
sfs_0fold_MTMINUS = []
sfs_t_MTMINUS = []
private_MTPLUS = []
private_MTMINUS = []
shared = []
fixed = []
Fst = []
GC4 = []

# Loop over all shared coding gametologs
alignment_files = glob.glob('../TranslationAligner/curated/*.fasta')
for a in alignment_files[:]:
    gene = a.split("/")[-1].split(".")[0]
    genes.append(gene)
    ness_id = a.split("/")[-1].lstrip(gene+".").rstrip(".quebec_mtPlus.fasta").rstrip(".quebec_mtMinus.fasta")
    print(a.split("/")[-1],gene, ness_id)
    aln = AlignIO.read(open(a), 'fasta')
    mtPlus_sampleIDs  = [s.id for s in aln if s.id.split("|")[0] in mtPlus_Genotypes]
    mtMinus_sampleIDs = [s.id for s in aln if s.id.split("|")[0] in mtMinus_Genotypes]
    mtPlus_coords.append(mtPlus_sampleIDs[0].split("|")[-1]) ## based on this kind of name e.g. "CC2935|mtMinus:105065-114642"
    mtMinus_coords.append(mtMinus_sampleIDs[0].split("|")[-1])
    degen_map = degenerate(special_consensus(aln))
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 12, degen_map, samples=None, start=None, end=None)
    ##### record ALL results
    alleles_ALL.append(sfst.alleles)
    sites_ALL.append(sfst.sites())
    invariant_ALL.append(sfst.invariant())
    variant_ALL.append(sfst.variant())
    theta_pi_0fold_ALL.append(sfs0.theta_pi())
    theta_pi_4fold_ALL.append(sfs4.theta_pi())
    theta_pi_total_ALL.append(sfst.theta_pi())
    theta_w_0fold_ALL.append(sfs0.theta_w())
    theta_w_4fold_ALL.append(sfs4.theta_w())
    theta_w_total_ALL.append(sfst.theta_w())
    sfs_4fold_ALL.append(sfs4.sfs)
    sfs_0fold_ALL.append(sfs0.sfs)
    sfs_t_ALL.append(sfst.sfs)
    ###################################
    ## MTPLUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtPlus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTPLUS.append(sfst.alleles)
    sites_MTPLUS.append(sfst.sites())
    invariant_MTPLUS.append(sfst.invariant())
    variant_MTPLUS.append(sfst.variant())
    theta_pi_0fold_MTPLUS.append(sfs0.theta_pi())
    theta_pi_4fold_MTPLUS.append(sfs4.theta_pi())
    theta_pi_total_MTPLUS.append(sfst.theta_pi())
    theta_w_0fold_MTPLUS.append(sfs0.theta_w())
    theta_w_4fold_MTPLUS.append(sfs4.theta_w())
    theta_w_total_MTPLUS.append(sfst.theta_w())
    sfs_4fold_MTPLUS.append(sfs4.sfs)
    sfs_0fold_MTPLUS.append(sfs0.sfs)
    sfs_t_MTPLUS.append(sfst.sfs)   
    ###################################
    ## MTMINUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtMinus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTMINUS.append(sfst.alleles)
    sites_MTMINUS.append(sfst.sites())
    invariant_MTMINUS.append(sfst.invariant())
    variant_MTMINUS.append(sfst.variant())
    theta_pi_0fold_MTMINUS.append(sfs0.theta_pi())
    theta_pi_4fold_MTMINUS.append(sfs4.theta_pi())
    theta_pi_total_MTMINUS.append(sfst.theta_pi())
    theta_w_0fold_MTMINUS.append(sfs0.theta_w())
    theta_w_4fold_MTMINUS.append(sfs4.theta_w())
    theta_w_total_MTMINUS.append(sfst.theta_w())
    sfs_4fold_MTMINUS.append(sfs4.sfs)
    sfs_0fold_MTMINUS.append(sfs0.sfs)
    sfs_t_MTMINUS.append(sfst.sfs)   
    ###################################
    ## population stats
    pop_stats= population_poly(aln, 12, degen_map, mtPlus_sampleIDs, mtMinus_sampleIDs, start=None, end=None)
    private_MTPLUS.append(  pop_stats['private1'])
    private_MTMINUS.append( pop_stats['private2'])
    shared.append(pop_stats['shared'])
    fixed.append(pop_stats['fixed'])
    Fst.append(pop_stats['fst'])
    #######
    ## GC_4fold
    GC4.append(len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'GCgc' and j =='4'])/
               len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'AaTtGCgc' and j =='4']))

#########################################################
#### DF the results #####################################
#########################################################
print(len(genes), len(ness_ids), len(mtPlus_coords), len(mtMinus_coords), len(alleles_ALL), len(sites_ALL), 
      len(invariant_ALL), len(variant_ALL), len(theta_pi_0fold_ALL), len(theta_pi_4fold_ALL), len(theta_pi_total_ALL), 
      len(theta_w_0fold_ALL), len(theta_w_4fold_ALL), len(theta_w_total_ALL), len(sfs_4fold_ALL), len(sfs_0fold_ALL), 
      len(sfs_t_ALL), len(alleles_MTPLUS), len(sites_MTPLUS), len(invariant_MTPLUS), len(variant_MTPLUS), 
      len(theta_pi_0fold_MTPLUS), len(theta_pi_4fold_MTPLUS), len(theta_pi_total_MTPLUS), len(theta_w_0fold_MTPLUS), 
      len(theta_w_4fold_MTPLUS), len(theta_w_total_MTPLUS), len(sfs_4fold_MTPLUS), len(sfs_0fold_MTPLUS), len(sfs_t_MTPLUS), 
      len(alleles_MTMINUS), len(sites_MTMINUS), len(invariant_MTMINUS), len(variant_MTMINUS), len(theta_pi_0fold_MTMINUS), 
      len(theta_pi_4fold_MTMINUS), len(theta_pi_total_MTMINUS), len(theta_w_0fold_MTMINUS), len(theta_w_4fold_MTMINUS), 
      len(theta_w_total_MTMINUS), len(sfs_4fold_MTMINUS), len(sfs_0fold_MTMINUS), len(sfs_t_MTMINUS), 
      len(private_MTPLUS), len(private_MTMINUS), len(shared), len(fixed), len(Fst), len(GC4))


shared_results = pd.DataFrame(
    {'gene' : genes,
#     'ness_id': ness_ids, # breaking df
     'mtPlus_coords' : mtPlus_coords,
     'mtMinus_coords' : mtMinus_coords,
     'alleles_ALL' : alleles_ALL, 
     'sites_ALL' : sites_ALL, 
     'invariant_ALL' : invariant_ALL, 
     'variant_ALL' : variant_ALL, 
     'theta_pi_0fold_ALL' : theta_pi_0fold_ALL, 
     'theta_pi_4fold_ALL' : theta_pi_4fold_ALL, 
     'theta_pi_total_ALL' : theta_pi_total_ALL, 
     'theta_w_0fold_ALL' : theta_w_0fold_ALL, 
     'theta_w_4fold_ALL' : theta_w_4fold_ALL, 
     'theta_w_total_ALL' : theta_w_total_ALL, 
     'sfs_4fold_ALL' : sfs_4fold_ALL, 
     'sfs_0fold_ALL' : sfs_0fold_ALL, 
     'sfs_t_ALL' : sfs_t_ALL,    
     'alleles_MTPLUS' : alleles_MTPLUS, 
     'sites_MTPLUS' : sites_MTPLUS, 
     'invariant_MTPLUS' : invariant_MTPLUS, 
     'variant_MTPLUS' : variant_MTPLUS, 
     'theta_pi_0fold_MTPLUS' : theta_pi_0fold_MTPLUS, 
     'theta_pi_4fold_MTPLUS' : theta_pi_4fold_MTPLUS, 
     'theta_pi_total_MTPLUS' : theta_pi_total_MTPLUS, 
     'theta_w_0fold_MTPLUS' : theta_w_0fold_MTPLUS, 
     'theta_w_4fold_MTPLUS' : theta_w_4fold_MTPLUS, 
     'theta_w_total_MTPLUS' : theta_w_total_MTPLUS, 
     'sfs_4fold_MTPLUS' : sfs_4fold_MTPLUS, 
     'sfs_0fold_MTPLUS' : sfs_0fold_MTPLUS, 
     'sfs_t_MTPLUS' : sfs_t_MTPLUS,         
     'alleles_MTMINUS' : alleles_MTMINUS, 
     'sites_MTMINUS' : sites_MTMINUS, 
     'invariant_MTMINUS' : invariant_MTMINUS, 
     'variant_MTMINUS' : variant_MTMINUS, 
     'theta_pi_0fold_MTMINUS' : theta_pi_0fold_MTMINUS, 
     'theta_pi_4fold_MTMINUS' : theta_pi_4fold_MTMINUS, 
     'theta_pi_total_MTMINUS' : theta_pi_total_MTMINUS, 
     'theta_w_0fold_MTMINUS' : theta_w_0fold_MTMINUS, 
     'theta_w_4fold_MTMINUS' : theta_w_4fold_MTMINUS, 
     'theta_w_total_MTMINUS' : theta_w_total_MTMINUS, 
     'sfs_4fold_MTMINUS' : sfs_4fold_MTMINUS, 
     'sfs_0fold_MTMINUS' : sfs_0fold_MTMINUS, 
     'sfs_t_MTMINUS' : sfs_t_MTMINUS, 
     'private_MTPLUS' : private_MTPLUS, 
     'private_MTMINUS' : private_MTMINUS, 
     'shared' : shared, 
     'fixed' : fixed, 
     'Fst' : Fst,
     'GC4' : GC4
    })     

EIF5Bb.quebec_all.aln.fasta EIF5Bb quebec_all.al
MT0829.quebec_all.aln.fasta MT0829 quebec_all.al
MT0828.quebec_all.aln.fasta MT0828 quebec_all.al
NIC7.quebec_all.aln (modified).fasta NIC7 quebec_all.aln (modified)
RFC4.quebec_all.aln.fasta RFC4 quebec_all.al
PKY1.quebec_all.aln.fasta PKY1 quebec_all.al
LPS1.quebec_all.aln (modified).fasta LPS1 quebec_all.aln (modified)
522915.quebec_all.aln.fasta 522915 quebec_all.al
522922.quebec_all.aln.fasta 522922 quebec_all.al
294752.quebec_all.aln.fasta 294752 quebec_all.al
522918.quebec_all.aln.fasta 522918 quebec_all.al
DLA3.quebec_all.aln.fasta DLA3 quebec_all.al
DRG1.quebec_all.aln.fasta DRG1 quebec_all.al
196073.quebec_all.aln.fasta 196073 quebec_all.al
HDH1.quebec_all.aln.fasta HDH1 quebec_all.al
NMDA1.quebec_all.aln.fasta NMDA1 quebec_all.al
GCSH.quebec_all.aln.fasta GCSH quebec_all.al
PSF2.quebec_all.aln.fasta PSF2 quebec_all.al
344092.quebec_all.aln.fasta 344092 quebec_all.al
196063.quebec_all.aln.fasta 196063 quebec_all.al
522914.quebe

In [35]:
shared_results.head(n=50)

,Fst,GC4,alleles_ALL,alleles_MTMINUS,alleles_MTPLUS,fixed,gene,invariant_ALL,invariant_MTMINUS,invariant_MTPLUS,...,theta_w_0fold_MTPLUS,theta_w_4fold_ALL,theta_w_4fold_MTMINUS,theta_w_4fold_MTPLUS,theta_w_total_ALL,theta_w_total_MTMINUS,theta_w_total_MTPLUS,variant_ALL,variant_MTMINUS,variant_MTPLUS
0,0.034259,0.866279,19,10,9,0,EIF5Bb,929,1080,991,...,0.000000,0.010146,0.008518,0.010012,0.006030,0.006111,0.005846,20,19,16
1,0.392467,0.809091,19,10,9,2,MT0829,1066,1075,1088,...,0.002562,0.018290,0.019369,0.005017,0.007323,0.006139,0.003351,28,19,10
2,0.466787,0.976190,19,10,9,0,MT0828,147,77,169,...,0.010316,0.016830,0.000000,0.017521,0.013005,0.000000,0.010573,7,0,5
3,0.131916,0.854801,19,10,9,2,NIC7,1443,1716,1654,...,0.000686,0.020373,0.016967,0.022688,0.007341,0.006670,0.008688,38,33,40
4,0.092847,0.798701,19,10,9,0,RFC4,641,724,651,...,0.000918,0.014619,0.020658,0.000000,0.004395,0.005763,0.000564,10,12,1
5,0.660817,0.779241,19,10,9,64,PKY1,5275,6158,7876,...,0.000722,0.010128,0.005515,0.001018,0.006882,0.003411,0.000839,130,60,18
6,0.583469,0.517949,19,10,9,5,LPS1,654,2257,1666,...,0.000338,0.017882,0.010941,0.001256,0.008490,0.005853,0.000441,20,38,2
7,0.315672,0.718750,19,10,9,0,522915,556,564,559,...,0.018642,0.007036,0.008692,0.009048,0.013251,0.011520,0.016953,27,19,27
8,0.166679,0.856517,19,10,9,0,522922,3972,3925,3980,...,0.002461,0.021451,0.021304,0.018869,0.009405,0.009381,0.007250,135,107,80
9,0.178789,0.821990,19,10,9,0,294752,2361,2362,2394,...,0.001882,0.015978,0.019741,0.005820,0.007666,0.009325,0.003350,65,64,22


In [36]:
# shared_results.to_csv('polymorphism_all.shared.csv', index = False)
shared_results.to_csv('polymorphism_all.20190225.shared.csv', index=False)

---
# mtLimited transcripts
---

In [71]:
# Target Genotype Lists
mtPlus_Genotypes = "CC2936 CC2937 CC3060 CC3064 CC3065 CC3068 CC3071 CC3076 CC3086".split()
mtMinus_Genotypes = "CC2935 CC2938 CC3059 CC3061 CC3062 CC3063 CC3073 CC3075 CC3079 CC3084".split()


#### STATS TO COLLECT for each gene
genes = []
mtPlus_coords = []
mtMinus_coords = []
alleles_ALL = []
sites_ALL = []
invariant_ALL = []
variant_ALL = []
theta_pi_0fold_ALL = []
theta_pi_4fold_ALL = []
theta_pi_total_ALL = []
theta_w_0fold_ALL = []
theta_w_4fold_ALL = []
theta_w_total_ALL = []
sfs_4fold_ALL = []
sfs_0fold_ALL = []
sfs_t_ALL = []
GC4 = []
# Loop over all shared coding gametologs
alignment_files = glob.glob('../VCF2FASTA/mtLimited/*.fasta')

for a in alignment_files[:]:
    print(a)
    gene = a.split("/")[-1].split(".")[0]
    mt_allele = a.split("/")[-1].split(".")[-2].split("_")[-1]
    print(gene, mt_allele)
    genes.append(gene)
    aln = AlignIO.read(open(a), 'fasta')
    if mt_allele == "mtPlus":
        mtPlus_sampleIDs  = [s.id for s in aln if s.id.split("|")[0] in mtPlus_Genotypes]
        mtPlus_coords.append(mtPlus_sampleIDs[0].split("|")[-1]) ## based on this kind of name e.g. "CC2935|mtMinus:105065-114642"
        mtMinus_coords.append('N/A')
    else:
        mtMinus_sampleIDs = [s.id for s in aln if s.id.split("|")[0] in mtMinus_Genotypes]
        mtMinus_coords.append(mtMinus_sampleIDs[0].split("|")[-1])
        mtPlus_coords.append("N/A")
    degen_map = degenerate(special_consensus(aln))
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 6, degen_map, samples=None, start=None, end=None)
    ##### record ALL results
    alleles_ALL.append(sfst.alleles)
    sites_ALL.append(sfst.sites())
    invariant_ALL.append(sfst.invariant())
    variant_ALL.append(sfst.variant())
    theta_pi_0fold_ALL.append(sfs0.theta_pi())
    theta_pi_4fold_ALL.append(sfs4.theta_pi())
    theta_pi_total_ALL.append(sfst.theta_pi())
    theta_w_0fold_ALL.append(sfs0.theta_w())
    theta_w_4fold_ALL.append(sfs4.theta_w())
    theta_w_total_ALL.append(sfst.theta_w())
    sfs_4fold_ALL.append(sfs4.sfs)
    sfs_0fold_ALL.append(sfs0.sfs)
    sfs_t_ALL.append(sfst.sfs)
    #######
    ## GC_4fold
    GC4.append(len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'GCgc' and j =='4'])/
               len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'AaTtGCgc' and j =='4']))


    
#########################################################
#### DF the results #####################################
#########################################################
mtLimited_results = pd.DataFrame(
    {'gene' : genes,
     'mtPlus_coords' : mtPlus_coords,
     'mtMinus_coords' : mtMinus_coords,
     'alleles_ALL' : alleles_ALL, 
     'sites_ALL' : sites_ALL, 
     'invariant_ALL' : invariant_ALL, 
     'variant_ALL' : variant_ALL, 
     'theta_pi_0fold_ALL' : theta_pi_0fold_ALL, 
     'theta_pi_4fold_ALL' : theta_pi_4fold_ALL, 
     'theta_pi_total_ALL' : theta_pi_total_ALL, 
     'theta_w_0fold_ALL' : theta_w_0fold_ALL, 
     'theta_w_4fold_ALL' : theta_w_4fold_ALL, 
     'theta_w_total_ALL' : theta_w_total_ALL, 
     'sfs_4fold_ALL' : sfs_4fold_ALL, 
     'sfs_0fold_ALL' : sfs_0fold_ALL, 
     'sfs_t_ALL' : sfs_t_ALL,    
     'GC4' : GC4
    })  
mtLimited_results.head()

../VCF2FASTA/mtLimited/MTP0428.26893319.quebec_mtPlus.fasta
MTP0428 mtPlus
../VCF2FASTA/mtLimited/MTA3.26894517.quebec_mtPlus.fasta
MTA3 mtPlus
../VCF2FASTA/mtLimited/MTA4.26893740.quebec_mtPlus.fasta
MTA4 mtPlus
../VCF2FASTA/mtLimited/MTA1.26893117.quebec_mtPlus.fasta
MTA1 mtPlus
../VCF2FASTA/mtLimited/INT1a.26894683.quebec_mtPlus.fasta
INT1a mtPlus
../VCF2FASTA/mtLimited/294708.26893164.quebec_mtPlus.fasta
294708 mtPlus
../VCF2FASTA/mtLimited/EZY2c.26894249.quebec_mtPlus.fasta
EZY2c mtPlus
../VCF2FASTA/mtLimited/MID.ADF43190.1.quebec_mtMinus.fasta
MID mtMinus
../VCF2FASTA/mtLimited/MTA5.26894560.quebec_mtPlus.fasta
MTA5 mtPlus
../VCF2FASTA/mtLimited/FUS1.26894229.quebec_mtPlus.fasta
FUS1 mtPlus
../VCF2FASTA/mtLimited/MTD1.ADF43193.1.quebec_mtMinus.fasta
MTD1 mtMinus
../VCF2FASTA/mtLimited/INT1b.26893202.quebec_mtPlus.fasta
INT1b mtPlus
../VCF2FASTA/mtLimited/EZY2d.26893701.quebec_mtPlus.fasta
EZY2d mtPlus


,GC4,alleles_ALL,gene,invariant_ALL,mtMinus_coords,mtPlus_coords,sfs_0fold_ALL,sfs_4fold_ALL,sfs_t_ALL,sites_ALL,theta_pi_0fold_ALL,theta_pi_4fold_ALL,theta_pi_total_ALL,theta_w_0fold_ALL,theta_w_4fold_ALL,theta_w_total_ALL,variant_ALL
0,0.717836,9,MTP0428,3110,N/A,chromosome_6:425999-429941,"[1941, 4, 3, 5, 0, 1, 5, 2, 5, 0]","[675, 2, 4, 1, 0, 0, 0, 1, 1, 0]","[3110, 8, 10, 7, 0, 1, 5, 4, 7, 0]",3152,0.004832,0.004548,0.004865,0.004679,0.004841,0.004903,42
1,0.756410,9,MTA3,380,N/A,chromosome_6:563911-564726,"[244, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[76, 1, 0, 0, 0, 0, 0, 0, 1, 0]","[380, 1, 0, 0, 0, 0, 0, 1, 1, 0]",383,0.000000,0.005698,0.002176,0.000000,0.009434,0.002882,3
2,0.880531,9,MTA4,1063,N/A,chromosome_6:548791-552710,"[673, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[221, 0, 2, 0, 0, 0, 0, 1, 2, 0]","[1063, 3, 2, 0, 0, 0, 0, 2, 3, 0]",1073,0.000000,0.007129,0.002692,0.000000,0.008140,0.003429,10
3,0.790323,9,MTA1,358,N/A,chromosome_6:559864-560244,"[221, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[62, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[358, 0, 0, 0, 0, 0, 0, 0, 0, 0]",358,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.849138,9,INT1a,1087,N/A,chromosome_6:740848-742579,"[702, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[232, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1087, 0, 0, 0, 0, 0, 0, 0, 1, 0]",1088,0.000316,0.000000,0.000204,0.000523,0.000000,0.000338,1


In [72]:
mtLimited_results.to_csv('polymorphism_all.mtLimited.csv', index = False)

# ch6 and random transcripts

In [ ]:
# Target Genotype Lists
mtPlus_Genotypes = "CC2936 CC2937 CC3060 CC3064 CC3065 CC3068 CC3071 CC3076 CC3086".split()
mtMinus_Genotypes = "CC2935 CC2938 CC3059 CC3061 CC3062 CC3063 CC3073 CC3075 CC3079 CC3084".split()


#### STATS TO COLLECT for each gene
genes = []
mtPlus_coords = []
mtMinus_coords = []
alleles_ALL = []
sites_ALL = []
invariant_ALL = []
variant_ALL = []
theta_pi_0fold_ALL = []
theta_pi_4fold_ALL = []
theta_pi_total_ALL = []
theta_w_0fold_ALL = []
theta_w_4fold_ALL = []
theta_w_total_ALL = []
sfs_4fold_ALL = []
sfs_0fold_ALL = []
sfs_t_ALL = []
alleles_MTPLUS = []
sites_MTPLUS = []
invariant_MTPLUS = []
variant_MTPLUS = []
theta_pi_0fold_MTPLUS = []
theta_pi_4fold_MTPLUS = []
theta_pi_total_MTPLUS = []
theta_w_0fold_MTPLUS = []
theta_w_4fold_MTPLUS = []
theta_w_total_MTPLUS = []
sfs_4fold_MTPLUS = []
sfs_0fold_MTPLUS = []
sfs_t_MTPLUS = []
alleles_MTMINUS = []
sites_MTMINUS = []
invariant_MTMINUS = []
variant_MTMINUS = []
theta_pi_0fold_MTMINUS = []
theta_pi_4fold_MTMINUS = []
theta_pi_total_MTMINUS = []
theta_w_0fold_MTMINUS = []
theta_w_4fold_MTMINUS = []
theta_w_total_MTMINUS = []
sfs_4fold_MTMINUS = []
sfs_0fold_MTMINUS = []
sfs_t_MTMINUS = []
private_MTPLUS = []
private_MTMINUS = []
shared = []
fixed = []
Fst = []
GC4 = []

# Loop over all shared coding gametologs
ch6_alignment_files = glob.glob('../VCF2FASTA/chromosome_6/*fasta')
random_alignment_files  =glob.glob('../VCF2FASTA/random_transcripts2/*fasta')

bc = 0
for a in ch6_alignment_files+random_alignment_files:
    bc+=1
    if bc %25 == 0:print(bc)
    gene = a.split("/")[-1].split(".")[0]
    genes.append(gene)
    aln = AlignIO.read(open(a), 'fasta')
    mtPlus_sampleIDs  = [s.id for s in aln if s.id.split("|")[0] in mtPlus_Genotypes]
    mtMinus_sampleIDs = [s.id for s in aln if s.id.split("|")[0] in mtMinus_Genotypes]
    mtPlus_coords.append(mtPlus_sampleIDs[0].split("|")[-1]) ## based on this kind of name e.g. "CC2935|mtMinus:105065-114642"
    mtMinus_coords.append(mtMinus_sampleIDs[0].split("|")[-1])
    degen_map = degenerate(special_consensus(aln))
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 12, degen_map, samples=None, start=None, end=None)
    ##### record ALL results
    alleles_ALL.append(sfst.alleles)
    sites_ALL.append(sfst.sites())
    invariant_ALL.append(sfst.invariant())
    variant_ALL.append(sfst.variant())
    theta_pi_0fold_ALL.append(sfs0.theta_pi())
    theta_pi_4fold_ALL.append(sfs4.theta_pi())
    theta_pi_total_ALL.append(sfst.theta_pi())
    theta_w_0fold_ALL.append(sfs0.theta_w())
    theta_w_4fold_ALL.append(sfs4.theta_w())
    theta_w_total_ALL.append(sfst.theta_w())
    sfs_4fold_ALL.append(sfs4.sfs)
    sfs_0fold_ALL.append(sfs0.sfs)
    sfs_t_ALL.append(sfst.sfs)
    ###################################
    ## MTPLUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtPlus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTPLUS.append(sfst.alleles)
    sites_MTPLUS.append(sfst.sites())
    invariant_MTPLUS.append(sfst.invariant())
    variant_MTPLUS.append(sfst.variant())
    theta_pi_0fold_MTPLUS.append(sfs0.theta_pi())
    theta_pi_4fold_MTPLUS.append(sfs4.theta_pi())
    theta_pi_total_MTPLUS.append(sfst.theta_pi())
    theta_w_0fold_MTPLUS.append(sfs0.theta_w())
    theta_w_4fold_MTPLUS.append(sfs4.theta_w())
    theta_w_total_MTPLUS.append(sfst.theta_w())
    sfs_4fold_MTPLUS.append(sfs4.sfs)
    sfs_0fold_MTPLUS.append(sfs0.sfs)
    sfs_t_MTPLUS.append(sfst.sfs)   
    ###################################
    ## MTMINUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtMinus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTMINUS.append(sfst.alleles)
    sites_MTMINUS.append(sfst.sites())
    invariant_MTMINUS.append(sfst.invariant())
    variant_MTMINUS.append(sfst.variant())
    theta_pi_0fold_MTMINUS.append(sfs0.theta_pi())
    theta_pi_4fold_MTMINUS.append(sfs4.theta_pi())
    theta_pi_total_MTMINUS.append(sfst.theta_pi())
    theta_w_0fold_MTMINUS.append(sfs0.theta_w())
    theta_w_4fold_MTMINUS.append(sfs4.theta_w())
    theta_w_total_MTMINUS.append(sfst.theta_w())
    sfs_4fold_MTMINUS.append(sfs4.sfs)
    sfs_0fold_MTMINUS.append(sfs0.sfs)
    sfs_t_MTMINUS.append(sfst.sfs)   
    ###################################
    ## population stats
    pop_stats= population_poly(aln, 12, degen_map, mtPlus_sampleIDs, mtMinus_sampleIDs, start=None, end=None)
    private_MTPLUS.append(  pop_stats['private1'])
    private_MTMINUS.append( pop_stats['private2'])
    shared.append(pop_stats['shared'])
    fixed.append(pop_stats['fixed'])
    Fst.append(pop_stats['fst'])
    #######
    ## GC_4fold
    try:
        gc =len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'GCgc' and j =='4'])/
            len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'AaTtGCgc' and j =='4'])
    except ZeroDivisionError:
        gc = 'N/A'
    GC4.append(gc)

#########################################################
#### DF the results #####################################
#########################################################
nonMTlocus_results = pd.DataFrame(
    {'gene' : genes,
     'mtPlus_coords' : mtPlus_coords,
     'mtMinus_coords' : mtMinus_coords,
     'alleles_ALL' : alleles_ALL, 
     'sites_ALL' : sites_ALL, 
     'invariant_ALL' : invariant_ALL, 
     'variant_ALL' : variant_ALL, 
     'theta_pi_0fold_ALL' : theta_pi_0fold_ALL, 
     'theta_pi_4fold_ALL' : theta_pi_4fold_ALL, 
     'theta_pi_total_ALL' : theta_pi_total_ALL, 
     'theta_w_0fold_ALL' : theta_w_0fold_ALL, 
     'theta_w_4fold_ALL' : theta_w_4fold_ALL, 
     'theta_w_total_ALL' : theta_w_total_ALL, 
     'sfs_4fold_ALL' : sfs_4fold_ALL, 
     'sfs_0fold_ALL' : sfs_0fold_ALL, 
     'sfs_t_ALL' : sfs_t_ALL,    
     'alleles_MTPLUS' : alleles_MTPLUS, 
     'sites_MTPLUS' : sites_MTPLUS, 
     'invariant_MTPLUS' : invariant_MTPLUS, 
     'variant_MTPLUS' : variant_MTPLUS, 
     'theta_pi_0fold_MTPLUS' : theta_pi_0fold_MTPLUS, 
     'theta_pi_4fold_MTPLUS' : theta_pi_4fold_MTPLUS, 
     'theta_pi_total_MTPLUS' : theta_pi_total_MTPLUS, 
     'theta_w_0fold_MTPLUS' : theta_w_0fold_MTPLUS, 
     'theta_w_4fold_MTPLUS' : theta_w_4fold_MTPLUS, 
     'theta_w_total_MTPLUS' : theta_w_total_MTPLUS, 
     'sfs_4fold_MTPLUS' : sfs_4fold_MTPLUS, 
     'sfs_0fold_MTPLUS' : sfs_0fold_MTPLUS, 
     'sfs_t_MTPLUS' : sfs_t_MTPLUS,         
     'alleles_MTMINUS' : alleles_MTMINUS, 
     'sites_MTMINUS' : sites_MTMINUS, 
     'invariant_MTMINUS' : invariant_MTMINUS, 
     'variant_MTMINUS' : variant_MTMINUS, 
     'theta_pi_0fold_MTMINUS' : theta_pi_0fold_MTMINUS, 
     'theta_pi_4fold_MTMINUS' : theta_pi_4fold_MTMINUS, 
     'theta_pi_total_MTMINUS' : theta_pi_total_MTMINUS, 
     'theta_w_0fold_MTMINUS' : theta_w_0fold_MTMINUS, 
     'theta_w_4fold_MTMINUS' : theta_w_4fold_MTMINUS, 
     'theta_w_total_MTMINUS' : theta_w_total_MTMINUS, 
     'sfs_4fold_MTMINUS' : sfs_4fold_MTMINUS, 
     'sfs_0fold_MTMINUS' : sfs_0fold_MTMINUS, 
     'sfs_t_MTMINUS' : sfs_t_MTMINUS, 
     'private_MTPLUS' : private_MTPLUS, 
     'private_MTMINUS' : private_MTMINUS, 
     'shared' : shared, 
     'fixed' : fixed, 
     'Fst' : Fst, 
     'GC4' : GC4
    }) 


nonMTlocus_results.head()


In [83]:
nonMTlocus_results.to_csv("polymorphism_all.NonMTLocus.csv", index=False)

---
# Gametologous Regions        
---


Here I go through the gametolog alignment that has been pinned together to map to the mtPlus allele 
and I calculate stats for silent sites including
 - 4 fold degneration
 - intron
 - intergenic

I also calculate a NOT silent bin which includes UTR/0-fold/2-fold

In [45]:
# Target Genotype Lists
mtPlus_Genotypes  = "CC2936 CC2937 CC3060 CC3064 CC3065 CC3068 CC3071 CC3076 CC3086".split()
mtMinus_Genotypes = "CC2935 CC2938 CC3059 CC3061 CC3062 CC3063 CC3073 CC3075 CC3079 CC3084".split()


#### STATS TO COLLECT 
genes = []
mtPlus_coords = []
mtMinus_coords = []
alleles_ALL = []
sites_ALL = []
invariant_ALL = []
variant_ALL = []
theta_pi_notSilent_ALL = []
theta_pi_silent_ALL = []
theta_pi_total_ALL = []
theta_w_notSilent_ALL = []
theta_w_silent_ALL = []
theta_w_total_ALL = []
sfs_silent_ALL = []
sfs_notSilent_ALL = []
sfs_t_ALL = []
alleles_MTPLUS = []
sites_MTPLUS = []
invariant_MTPLUS = []
variant_MTPLUS = []
theta_pi_notSilent_MTPLUS = []
theta_pi_silent_MTPLUS = []
theta_pi_total_MTPLUS = []
theta_w_notSilent_MTPLUS = []
theta_w_silent_MTPLUS = []
theta_w_total_MTPLUS = []
sfs_silent_MTPLUS = []
sfs_notSilent_MTPLUS = []
sfs_t_MTPLUS = []
alleles_MTMINUS = []
sites_MTMINUS = []
invariant_MTMINUS = []
variant_MTMINUS = []
theta_pi_notSilent_MTMINUS = []
theta_pi_silent_MTMINUS = []
theta_pi_total_MTMINUS = []
theta_w_notSilent_MTMINUS = []
theta_w_silent_MTMINUS = []
theta_w_total_MTMINUS = []
sfs_silent_MTMINUS = []
sfs_notSilent_MTMINUS = []
sfs_t_MTMINUS = []
private_MTPLUS = []
private_MTMINUS = []
shared = []
fixed = []
Fst = []
GCsilent = []


# To calculate silent/vs non-silent stats I can manipulate the degen map as follows:
# - 4 = all silent sites -intron, intergenic, 4-fold
# I need to use the annotation table for the region   
# 298298-943474 (0-based python style)
import annotation_table, pysam
annotation_table_file = "/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/concatenated_GFF/annotation_table.txt.gz"
at = pysam.Tabixfile(annotation_table_file)
special_degen_map = ''
for l in  at.fetch('chromosome_6',298298,943474): #note used 0-based non inclusive coords
    a = annotation_table.annotation_line(l)
    if a.fold4 =='1' or a.intergenic =='1' or a.intronic =='1': # all silent sites are marked as 4-fold
        special_degen_map+='4'
    else:
        special_degen_map+='0'

wtfs = glob.glob("../../../data/aligned-fastas/mt_aligned_all.fasta")
for a in wtfs:
    gene = a.split("/")[-1].split(".")[0]
    genes.append(gene)
    aln = AlignIO.read(open(a), 'fasta')
    mtPlus_sampleIDs  = [s.id for s in aln if s.id.split("|")[0] in mtPlus_Genotypes]
    mtMinus_sampleIDs = [s.id for s in aln if s.id.split("|")[0] in mtMinus_Genotypes]
    mtPlus_coords.append('gametologs_pluscoords') 
    mtMinus_coords.append('gametologs_pluscoords')
    degen_map = special_degen_map
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 12, degen_map, samples=None, start=None, end=None)
    ##### record ALL results
    alleles_ALL.append(sfst.alleles)
    sites_ALL.append(sfst.sites())
    invariant_ALL.append(sfst.invariant())
    variant_ALL.append(sfst.variant())
    theta_pi_notSilent_ALL.append(sfs0.theta_pi())
    theta_pi_silent_ALL.append(sfs4.theta_pi())
    theta_pi_total_ALL.append(sfst.theta_pi())
    theta_w_notSilent_ALL.append(sfs0.theta_w())
    theta_w_silent_ALL.append(sfs4.theta_w())
    theta_w_total_ALL.append(sfst.theta_w())
    sfs_silent_ALL.append(sfs4.sfs)
    sfs_notSilent_ALL.append(sfs0.sfs)
    sfs_t_ALL.append(sfst.sfs)
    ###################################
    ## MTPLUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtPlus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTPLUS.append(sfst.alleles)
    sites_MTPLUS.append(sfst.sites())
    invariant_MTPLUS.append(sfst.invariant())
    variant_MTPLUS.append(sfst.variant())
    theta_pi_notSilent_MTPLUS.append(sfs0.theta_pi())
    theta_pi_silent_MTPLUS.append(sfs4.theta_pi())
    theta_pi_total_MTPLUS.append(sfst.theta_pi())
    theta_w_notSilent_MTPLUS.append(sfs0.theta_w())
    theta_w_silent_MTPLUS.append(sfs4.theta_w())
    theta_w_total_MTPLUS.append(sfst.theta_w())
    sfs_silent_MTPLUS.append(sfs4.sfs)
    sfs_notSilent_MTPLUS.append(sfs0.sfs)
    sfs_t_MTPLUS.append(sfst.sfs)   
    ###################################
    ## MTMINUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtMinus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTMINUS.append(sfst.alleles)
    sites_MTMINUS.append(sfst.sites())
    invariant_MTMINUS.append(sfst.invariant())
    variant_MTMINUS.append(sfst.variant())
    theta_pi_notSilent_MTMINUS.append(sfs0.theta_pi())
    theta_pi_silent_MTMINUS.append(sfs4.theta_pi())
    theta_pi_total_MTMINUS.append(sfst.theta_pi())
    theta_w_notSilent_MTMINUS.append(sfs0.theta_w())
    theta_w_silent_MTMINUS.append(sfs4.theta_w())
    theta_w_total_MTMINUS.append(sfst.theta_w())
    sfs_silent_MTMINUS.append(sfs4.sfs)
    sfs_notSilent_MTMINUS.append(sfs0.sfs)
    sfs_t_MTMINUS.append(sfst.sfs)   
    ###################################
    ## population stats
    pop_stats= population_poly(aln, 12, degen_map, mtPlus_sampleIDs, mtMinus_sampleIDs, start=None, end=None)
    private_MTPLUS.append(  pop_stats['private1'])
    private_MTMINUS.append( pop_stats['private2'])
    shared.append(pop_stats['shared'])
    fixed.append(pop_stats['fixed'])
    Fst.append(pop_stats['fst'])
    #######
    ## GC_silent
    GCsilent.append(len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'GCgc' and j =='4'])/
               len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'AaTtGCgc' and j =='4']))

    
#########################################################
#### DF the results #####################################
#########################################################
gametolog_results = pd.DataFrame(
    {'gene' : genes,
     'mtPlus_coords' : mtPlus_coords,
     'mtMinus_coords' : mtMinus_coords,
     'alleles_ALL' : alleles_ALL, 
     'sites_ALL' : sites_ALL, 
     'invariant_ALL' : invariant_ALL, 
     'variant_ALL' : variant_ALL, 
     'theta_pi_notSilent_ALL' : theta_pi_notSilent_ALL, 
     'theta_pi_silent_ALL' : theta_pi_silent_ALL, 
     'theta_pi_total_ALL' : theta_pi_total_ALL, 
     'theta_w_notSilent_ALL' : theta_w_notSilent_ALL, 
     'theta_w_silent_ALL' : theta_w_silent_ALL, 
     'theta_w_total_ALL' : theta_w_total_ALL, 
     'sfs_silent_ALL' : sfs_silent_ALL, 
     'sfs_notSilent_ALL' : sfs_notSilent_ALL, 
     'sfs_t_ALL' : sfs_t_ALL,    
     'alleles_MTPLUS' : alleles_MTPLUS, 
     'sites_MTPLUS' : sites_MTPLUS, 
     'invariant_MTPLUS' : invariant_MTPLUS, 
     'variant_MTPLUS' : variant_MTPLUS, 
     'theta_pi_notSilent_MTPLUS' : theta_pi_notSilent_MTPLUS, 
     'theta_pi_silent_MTPLUS' : theta_pi_silent_MTPLUS, 
     'theta_pi_total_MTPLUS' : theta_pi_total_MTPLUS, 
     'theta_w_notSilent_MTPLUS' : theta_w_notSilent_MTPLUS, 
     'theta_w_silent_MTPLUS' : theta_w_silent_MTPLUS, 
     'theta_w_total_MTPLUS' : theta_w_total_MTPLUS, 
     'sfs_silent_MTPLUS' : sfs_silent_MTPLUS, 
     'sfs_notSilent_MTPLUS' : sfs_notSilent_MTPLUS, 
     'sfs_t_MTPLUS' : sfs_t_MTPLUS,         
     'alleles_MTMINUS' : alleles_MTMINUS, 
     'sites_MTMINUS' : sites_MTMINUS, 
     'invariant_MTMINUS' : invariant_MTMINUS, 
     'variant_MTMINUS' : variant_MTMINUS, 
     'theta_pi_notSilent_MTMINUS' : theta_pi_notSilent_MTMINUS, 
     'theta_pi_silent_MTMINUS' : theta_pi_silent_MTMINUS, 
     'theta_pi_total_MTMINUS' : theta_pi_total_MTMINUS, 
     'theta_w_notSilent_MTMINUS' : theta_w_notSilent_MTMINUS, 
     'theta_w_silent_MTMINUS' : theta_w_silent_MTMINUS, 
     'theta_w_total_MTMINUS' : theta_w_total_MTMINUS, 
     'sfs_silent_MTMINUS' : sfs_silent_MTMINUS, 
     'sfs_notSilent_MTMINUS' : sfs_notSilent_MTMINUS, 
     'sfs_t_MTMINUS' : sfs_t_MTMINUS, 
     'private_MTPLUS' : private_MTPLUS, 
     'private_MTMINUS' : private_MTMINUS, 
     'shared' : shared, 
     'fixed' : fixed, 
     'Fst' : Fst, 
     'GCsilent' : GCsilent
    })


gametolog_results.to_csv("polymorphism_all.gametolog.csv", index=False)
gametolog_results.head()

,Fst,GCsilent,alleles_ALL,alleles_MTMINUS,alleles_MTPLUS,fixed,gene,invariant_ALL,invariant_MTMINUS,invariant_MTPLUS,...,theta_w_notSilent_MTPLUS,theta_w_silent_ALL,theta_w_silent_MTMINUS,theta_w_silent_MTPLUS,theta_w_total_ALL,theta_w_total_MTMINUS,theta_w_total_MTPLUS,variant_ALL,variant_MTMINUS,variant_MTPLUS
0,0.429806,0.657755,19,10,9,5525,mt_aligned_all,395496,400640,415016,...,0.003757,0.016504,0.012225,0.007162,0.013267,0.009707,0.005548,19231,11313,6354


---
# Gametolog w Intron and Intergenic vs other
---


Here I go through the gametolog alignment that has been pinned together to map to the mtPlus allele 
and I calculate stats for intron and intergenic sites and total sites

** Although 0-fold is reported there are no data in this category because it was masked **


In [43]:
#### STATS TO COLLECT 
genes = []
mtPlus_coords = []
mtMinus_coords = []
alleles_ALL = []
sites_ALL = []
invariant_ALL = []
variant_ALL = []
theta_pi_0fold_ALL = []
theta_pi_IandI_ALL = []
theta_pi_total_ALL = []
theta_w_0fold_ALL = []
theta_w_IandI_ALL = []
theta_w_total_ALL = []
sfs_IandI_ALL = []
sfs_0fold_ALL = []
sfs_t_ALL = []
alleles_MTPLUS = []
sites_MTPLUS = []
invariant_MTPLUS = []
variant_MTPLUS = []
theta_pi_0fold_MTPLUS = []
theta_pi_IandI_MTPLUS = []
theta_pi_total_MTPLUS = []
theta_w_0fold_MTPLUS = []
theta_w_IandI_MTPLUS = []
theta_w_total_MTPLUS = []
sfs_IandI_MTPLUS = []
sfs_0fold_MTPLUS = []
sfs_t_MTPLUS = []
alleles_MTMINUS = []
sites_MTMINUS = []
invariant_MTMINUS = []
variant_MTMINUS = []
theta_pi_0fold_MTMINUS = []
theta_pi_IandI_MTMINUS = []
theta_pi_total_MTMINUS = []
theta_w_0fold_MTMINUS = []
theta_w_IandI_MTMINUS = []
theta_w_total_MTMINUS = []
sfs_IandI_MTMINUS = []
sfs_0fold_MTMINUS = []
sfs_t_MTMINUS = []
private_MTPLUS = []
private_MTMINUS = []
shared = []
fixed = []
Fst = []
GC_IandI = []


# To calculate silent/vs non-silent stats I can manipulate the degen map as follows:
# - 4 = all silent sites -intron, intergenic, 4-fold
# I need to use the annotation table for the region   
# 298298-943474 (0-based python style)
import annotation_table, pysam
annotation_table_file = "/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/concatenated_GFF/annotation_table.txt.gz"
at = pysam.Tabixfile(annotation_table_file)
special_degen_map = ''
for l in  at.fetch('chromosome_6',298298,943474): #note used 0-based non inclusive coords
    a = annotation_table.annotation_line(l)
    if a.intergenic =='1' or a.intronic =='1':
        special_degen_map+='4'
    else:
        special_degen_map+='2' # this effectively eliminates 0-fold stats

wtfs = glob.glob("../../../data/aligned-fastas/mt_aligned_all.fasta")
for a in wtfs:
    gene = a.split("/")[-1].split(".")[0]
    genes.append(gene)
    aln = AlignIO.read(open(a), 'fasta')
    mtPlus_sampleIDs  = [s.id for s in aln if s.id.split("|")[0] in mtPlus_Genotypes]
    mtMinus_sampleIDs = [s.id for s in aln if s.id.split("|")[0] in mtMinus_Genotypes]
    mtPlus_coords.append('gametologs_pluscoords_intron_intergenic') 
    mtMinus_coords.append('gametologs_pluscoords_intron_intergenic')
    degen_map = special_degen_map
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 12, degen_map, samples=None, start=None, end=None)
    ##### record ALL results
    alleles_ALL.append(sfst.alleles)
    sites_ALL.append(sfst.sites())
    invariant_ALL.append(sfst.invariant())
    variant_ALL.append(sfst.variant())
    theta_pi_0fold_ALL.append(sfs0.theta_pi())
    theta_pi_IandI_ALL.append(sfs4.theta_pi())
    theta_pi_total_ALL.append(sfst.theta_pi())
    theta_w_0fold_ALL.append(sfs0.theta_w())
    theta_w_IandI_ALL.append(sfs4.theta_w())
    theta_w_total_ALL.append(sfst.theta_w())
    sfs_IandI_ALL.append(sfs4.sfs)
    sfs_0fold_ALL.append(sfs0.sfs)
    sfs_t_ALL.append(sfst.sfs)
    ###################################
    ## MTPLUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtPlus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTPLUS.append(sfst.alleles)
    sites_MTPLUS.append(sfst.sites())
    invariant_MTPLUS.append(sfst.invariant())
    variant_MTPLUS.append(sfst.variant())
    theta_pi_0fold_MTPLUS.append(sfs0.theta_pi())
    theta_pi_IandI_MTPLUS.append(sfs4.theta_pi())
    theta_pi_total_MTPLUS.append(sfst.theta_pi())
    theta_w_0fold_MTPLUS.append(sfs0.theta_w())
    theta_w_IandI_MTPLUS.append(sfs4.theta_w())
    theta_w_total_MTPLUS.append(sfst.theta_w())
    sfs_IandI_MTPLUS.append(sfs4.sfs)
    sfs_0fold_MTPLUS.append(sfs0.sfs)
    sfs_t_MTPLUS.append(sfst.sfs)   
    ###################################
    ## MTMINUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtMinus_sampleIDs, start=None, end=None)
    ##### record ALL results
    alleles_MTMINUS.append(sfst.alleles)
    sites_MTMINUS.append(sfst.sites())
    invariant_MTMINUS.append(sfst.invariant())
    variant_MTMINUS.append(sfst.variant())
    theta_pi_0fold_MTMINUS.append(sfs0.theta_pi())
    theta_pi_IandI_MTMINUS.append(sfs4.theta_pi())
    theta_pi_total_MTMINUS.append(sfst.theta_pi())
    theta_w_0fold_MTMINUS.append(sfs0.theta_w())
    theta_w_IandI_MTMINUS.append(sfs4.theta_w())
    theta_w_total_MTMINUS.append(sfst.theta_w())
    sfs_IandI_MTMINUS.append(sfs4.sfs)
    sfs_0fold_MTMINUS.append(sfs0.sfs)
    sfs_t_MTMINUS.append(sfst.sfs)   
    ###################################
    ## population stats
    pop_stats= population_poly(aln, 12, degen_map, mtPlus_sampleIDs, mtMinus_sampleIDs, start=None, end=None)
    private_MTPLUS.append(  pop_stats['private1'])
    private_MTMINUS.append( pop_stats['private2'])
    shared.append(pop_stats['shared'])
    fixed.append(pop_stats['fixed'])
    Fst.append(pop_stats['fst'])
    #######
    ## GC_IandI
    GC_IandI.append(len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'GCgc' and j =='4'])/
               len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'AaTtGCgc' and j =='4']))

    
#########################################################
#### DF the results #####################################
#########################################################
gametolog_intron_intergenic_results = pd.DataFrame(
    {'gene' : genes,
     'mtPlus_coords' : mtPlus_coords,
     'mtMinus_coords' : mtMinus_coords,
     'alleles_ALL' : alleles_ALL, 
     'sites_ALL' : sites_ALL, 
     'invariant_ALL' : invariant_ALL, 
     'variant_ALL' : variant_ALL, 
     'theta_pi_0fold_ALL' : theta_pi_0fold_ALL, 
     'theta_pi_IandI_ALL' : theta_pi_IandI_ALL, 
     'theta_pi_total_ALL' : theta_pi_total_ALL, 
     'theta_w_0fold_ALL' : theta_w_0fold_ALL, 
     'theta_w_IandI_ALL' : theta_w_IandI_ALL, 
     'theta_w_total_ALL' : theta_w_total_ALL, 
     'sfs_IandI_ALL' : sfs_IandI_ALL, 
     'sfs_0fold_ALL' : sfs_0fold_ALL, 
     'sfs_t_ALL' : sfs_t_ALL,    
     'alleles_MTPLUS' : alleles_MTPLUS, 
     'sites_MTPLUS' : sites_MTPLUS, 
     'invariant_MTPLUS' : invariant_MTPLUS, 
     'variant_MTPLUS' : variant_MTPLUS, 
     'theta_pi_0fold_MTPLUS' : theta_pi_0fold_MTPLUS, 
     'theta_pi_IandI_MTPLUS' : theta_pi_IandI_MTPLUS, 
     'theta_pi_total_MTPLUS' : theta_pi_total_MTPLUS, 
     'theta_w_0fold_MTPLUS' : theta_w_0fold_MTPLUS, 
     'theta_w_IandI_MTPLUS' : theta_w_IandI_MTPLUS, 
     'theta_w_total_MTPLUS' : theta_w_total_MTPLUS, 
     'sfs_IandI_MTPLUS' : sfs_IandI_MTPLUS, 
     'sfs_0fold_MTPLUS' : sfs_0fold_MTPLUS, 
     'sfs_t_MTPLUS' : sfs_t_MTPLUS,         
     'alleles_MTMINUS' : alleles_MTMINUS, 
     'sites_MTMINUS' : sites_MTMINUS, 
     'invariant_MTMINUS' : invariant_MTMINUS, 
     'variant_MTMINUS' : variant_MTMINUS, 
     'theta_pi_0fold_MTMINUS' : theta_pi_0fold_MTMINUS, 
     'theta_pi_IandI_MTMINUS' : theta_pi_IandI_MTMINUS, 
     'theta_pi_total_MTMINUS' : theta_pi_total_MTMINUS, 
     'theta_w_0fold_MTMINUS' : theta_w_0fold_MTMINUS, 
     'theta_w_IandI_MTMINUS' : theta_w_IandI_MTMINUS, 
     'theta_w_total_MTMINUS' : theta_w_total_MTMINUS, 
     'sfs_IandI_MTMINUS' : sfs_IandI_MTMINUS, 
     'sfs_0fold_MTMINUS' : sfs_0fold_MTMINUS, 
     'sfs_t_MTMINUS' : sfs_t_MTMINUS, 
     'private_MTPLUS' : private_MTPLUS, 
     'private_MTMINUS' : private_MTMINUS, 
     'shared' : shared, 
     'fixed' : fixed, 
     'Fst' : Fst, 
     'GC_IandI' : GC_IandI
    })


gametolog_intron_intergenic_results.to_csv("polymorphism_all.gametolog_intron_intergenic.csv", index=False)
gametolog_intron_intergenic_results.head()

,Fst,GC_IandI,alleles_ALL,alleles_MTMINUS,alleles_MTPLUS,fixed,gene,invariant_ALL,invariant_MTMINUS,invariant_MTPLUS,...,theta_w_0fold_MTPLUS,theta_w_IandI_ALL,theta_w_IandI_MTMINUS,theta_w_IandI_MTPLUS,theta_w_total_ALL,theta_w_total_MTMINUS,theta_w_total_MTPLUS,variant_ALL,variant_MTMINUS,variant_MTPLUS
0,0.429806,0.633101,19,10,9,5525,mt_aligned_all,395496,400640,415016,...,NaN,0.017022,0.012209,0.007242,0.013267,0.009707,0.005548,19231,11313,6354


In [40]:
print(gametolog_results.private_MTMINUS[0],
gametolog_results.private_MTPLUS[0],
gametolog_results.fixed[0],
gametolog_results.shared[0],)

7299 2237 5525 4170


---
# Non-gametologous regions
---

In [44]:
# Target Genotype Lists
mtPlus_Genotypes = "CC2936 CC2937 CC3060 CC3064 CC3065 CC3068 CC3071 CC3076 CC3086".split()
mtMinus_Genotypes = "CC2935 CC2938 CC3059 CC3061 CC3062 CC3063 CC3073 CC3075 CC3079 CC3084".split()


#### STATS TO COLLECT for each gene
genes = []
mtPlus_coords = []
mtMinus_coords = []
alleles_ALL = []
sites_ALL = []
invariant_ALL = []
variant_ALL = []
theta_pi_total_ALL = []
theta_w_total_ALL = []
sfs_t_ALL = []
# Loop over all shared coding gametologs
alignment_files = glob.glob("../../../data/aligned-fastas/*non_gametolog.fasta")
print(alignment_files)

for a in alignment_files[:]:
    !head $a -n 1 
#     print(a)
#     gene = a.split("/")[-1].split(".")[0]
#     mt_allele = a.split("/")[-1].split(".")[-2].split("_")[-1]
#     print(gene, mt_allele)
#     genes.append(gene)
#     aln = AlignIO.read(open(a), 'fasta')
#     if mt_allele == "mtPlus":
#         mtPlus_sampleIDs  = [s.id for s in aln if s.id.split("|")[0] in mtPlus_Genotypes]
#         mtPlus_coords.append(mtPlus_sampleIDs[0].split("|")[-1]) ## based on this kind of name e.g. "CC2935|mtMinus:105065-114642"
#         mtMinus_coords.append('N/A')
#     else:
#         mtMinus_sampleIDs = [s.id for s in aln if s.id.split("|")[0] in mtMinus_Genotypes]
#         mtMinus_coords.append(mtMinus_sampleIDs[0].split("|")[-1])
#         mtPlus_coords.append("N/A")
#     degen_map = degenerate(special_consensus(aln))
#     sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 6, degen_map, samples=None, start=None, end=None)
#     ##### record ALL results
#     alleles_ALL.append(sfst.alleles)
#     sites_ALL.append(sfst.sites())
#     invariant_ALL.append(sfst.invariant())
#     variant_ALL.append(sfst.variant())
#     theta_pi_0fold_ALL.append(sfs0.theta_pi())
#     theta_pi_4fold_ALL.append(sfs4.theta_pi())
#     theta_pi_total_ALL.append(sfst.theta_pi())
#     theta_w_0fold_ALL.append(sfs0.theta_w())
#     theta_w_4fold_ALL.append(sfs4.theta_w())
#     theta_w_total_ALL.append(sfst.theta_w())
#     sfs_4fold_ALL.append(sfs4.sfs)
#     sfs_0fold_ALL.append(sfs0.sfs)
#     sfs_t_ALL.append(sfst.sfs)
#     #######
#     ## GC_4fold
#      GC4.append(len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'GCgc' and j =='4'])/
#                 len([1 for i,j in zip(special_consensus(aln),degen_map) if i in 'AaTtGCgc' and j =='4']))
   
# #########################################################
# #### DF the results #####################################
# #########################################################
# mtLimited_results = pd.DataFrame(
#     {'gene' : genes,
#      'mtPlus_coords' : mtPlus_coords,
#      'mtMinus_coords' : mtMinus_coords,
#      'alleles_ALL' : alleles_ALL, 
#      'sites_ALL' : sites_ALL, 
#      'invariant_ALL' : invariant_ALL, 
#      'variant_ALL' : variant_ALL, 
#      'theta_pi_0fold_ALL' : theta_pi_0fold_ALL, 
#      'theta_pi_4fold_ALL' : theta_pi_4fold_ALL, 
#      'theta_pi_total_ALL' : theta_pi_total_ALL, 
#      'theta_w_0fold_ALL' : theta_w_0fold_ALL, 
#      'theta_w_4fold_ALL' : theta_w_4fold_ALL, 
#      'theta_w_total_ALL' : theta_w_total_ALL, 
#      'sfs_4fold_ALL' : sfs_4fold_ALL, 
#      'sfs_0fold_ALL' : sfs_0fold_ALL, 
#      'sfs_t_ALL' : sfs_t_ALL,    
#      'GC4' : GC4
#     })  
# mtLimited_results.head()

['../../../data/aligned-fastas/plus_non_gametolog.fasta', '../../../data/aligned-fastas/minus_non_gametolog.fasta']
>CC3076
>CC3061


---
# Windowed Gametolog #AHMED
---

In [20]:
from tqdm import tqdm
# Target Genotype Lists
mtPlus_Genotypes  = "CC2936 CC2937 CC3060 CC3064 CC3065 CC3068 CC3071 CC3076 CC3086".split()
mtMinus_Genotypes = "CC2935 CC2938 CC3059 CC3061 CC3062 CC3063 CC3073 CC3075 CC3079 CC3084".split()


#### STATS TO COLLECT 
windows = []
start_coords = []
end_coords = []
alleles_ALL = []
sites_ALL = []
invariant_ALL = []
variant_ALL = []
theta_pi_notSilent_ALL = []
theta_pi_silent_ALL = []
theta_pi_total_ALL = []
theta_w_notSilent_ALL = []
theta_w_silent_ALL = []
theta_w_total_ALL = []
sfs_silent_ALL = []
sfs_notSilent_ALL = []
sfs_t_ALL = []
alleles_MTPLUS = []
sites_MTPLUS = []
invariant_MTPLUS = []
variant_MTPLUS = []
theta_pi_notSilent_MTPLUS = []
theta_pi_silent_MTPLUS = []
theta_pi_total_MTPLUS = []
theta_w_notSilent_MTPLUS = []
theta_w_silent_MTPLUS = []
theta_w_total_MTPLUS = []
sfs_silent_MTPLUS = []
sfs_notSilent_MTPLUS = []
sfs_t_MTPLUS = []
alleles_MTMINUS = []
sites_MTMINUS = []
invariant_MTMINUS = []
variant_MTMINUS = []
theta_pi_notSilent_MTMINUS = []
theta_pi_silent_MTMINUS = []
theta_pi_total_MTMINUS = []
theta_w_notSilent_MTMINUS = []
theta_w_silent_MTMINUS = []
theta_w_total_MTMINUS = []
sfs_silent_MTMINUS = []
sfs_notSilent_MTMINUS = []
sfs_t_MTMINUS = []
private_MTPLUS = []
private_MTMINUS = []
shared = []
fixed = []
Fst = []
GCsilent = []


# To calculate silent/vs non-silent stats I can manipulate the degen map as follows:
# - 4 = all silent sites -intron, intergenic, 4-fold
# I need to use the annotation table for the region   
# 298298-943474 (0-based python style)

import annotation_table, pysam
annotation_table_file = "/scratch/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/concatenated_GFF/annotation_table.txt.gz"
at = pysam.Tabixfile(annotation_table_file)
special_degen_map = ''
for l in  at.fetch('chromosome_6',298298,943474): #note used 0-based non inclusive coords
    a = annotation_table.annotation_line(l)
    if a.fold4 =='1' or a.intergenic =='1' or a.intronic =='1': # all silent sites are marked as 4-fold
        special_degen_map+='4'
    else:
        special_degen_map+='0'

a = ("../../../data/aligned-fastas/mt_aligned_all.fasta")
aln = AlignIO.read(open(a), 'fasta')

#Ahmed Windows
# Add windows
window_coords = [] # this should be [start1, end1], [start2, end2]... in python coords

def get_genomic_coords(start, end, windowsize, offset):
    ''' 
    returns genomic start and end of entire region,
    rounded to match windowsize
    start: alignment python index start (0)
    end: alignment python index end
    windowsize: desired windowsize
    offset: what genomic position is at alignment index 0?
    '''
    genomic_start = round((start + offset) / windowsize) * windowsize
    genomic_end = (round((end + offset) / windowsize) * windowsize) + windowsize
    if genomic_start > offset:
        genomic_start = genomic_start - windowsize
    if genomic_end - windowsize > offset + end:
        genomic_end = genomic_end - windowsize
    return genomic_start, genomic_end

def get_window_coords(start, end, windowsize, offset):
    ''' 
    returns python index windows
    start: alignment python index start (ie 0)
    end: alignment python index end
    windowsize: desired windowsize
    offset: what genomic position is at alignment index 0?
    '''
    genomic_start, genomic_end = get_genomic_coords(start, end, windowsize, offset)
    genomic_windows = list(range(genomic_start, genomic_end + windowsize, windowsize))
    aln_offset = windowsize - (offset - genomic_windows[0])
    window_coords_all = [0] + [aln_offset + (windowsize * i) for i in range(len(genomic_windows) - 1)]
    window_coords_all[-1] = end # avoid IndexError in final window
    window_coords_nested = [window_coords_all[i:i+2] for i in range(len(window_coords_all) - 1)]
    return window_coords_nested
    
start = 0
end = 645176
windowsize = 1000 # to match zns file
offset = 298298
windows_coords = get_window_coords(start, end, windowsize, offset)

for w in tqdm(windows_coords):
    aln_start, aln_end = w
    window = "%i-%i" %(aln_start, aln_end)
    windows.append(window)
    mtPlus_sampleIDs  = [s.id for s in aln if s.id.split("|")[0] in mtPlus_Genotypes]
    mtMinus_sampleIDs = [s.id for s in aln if s.id.split("|")[0] in mtMinus_Genotypes]
    genome_start = aln_start + offset # globally set offset is 298298
    genome_end = aln_end + offset
    start_coords.append(genome_start) 
    end_coords.append(genome_end)
    degen_map = special_degen_map
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 12, degen_map, samples=None, start=aln_start, end=aln_end)
    ##### record ALL results
    alleles_ALL.append(sfst.alleles)
    sites_ALL.append(sfst.sites())
    invariant_ALL.append(sfst.invariant())
    variant_ALL.append(sfst.variant())
    theta_pi_notSilent_ALL.append(sfs0.theta_pi())
    theta_pi_silent_ALL.append(sfs4.theta_pi())
    theta_pi_total_ALL.append(sfst.theta_pi())
    theta_w_notSilent_ALL.append(sfs0.theta_w())
    theta_w_silent_ALL.append(sfs4.theta_w())
    theta_w_total_ALL.append(sfst.theta_w())
    sfs_silent_ALL.append(sfs4.sfs)
    sfs_notSilent_ALL.append(sfs0.sfs)
    sfs_t_ALL.append(sfst.sfs)
    ###################################
    ## MTPLUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtPlus_sampleIDs, start=aln_start, end=aln_end)
    ##### record ALL results
    alleles_MTPLUS.append(sfst.alleles)
    sites_MTPLUS.append(sfst.sites())
    invariant_MTPLUS.append(sfst.invariant())
    variant_MTPLUS.append(sfst.variant())
    theta_pi_notSilent_MTPLUS.append(sfs0.theta_pi())
    theta_pi_silent_MTPLUS.append(sfs4.theta_pi())
    theta_pi_total_MTPLUS.append(sfst.theta_pi())
    theta_w_notSilent_MTPLUS.append(sfs0.theta_w())
    theta_w_silent_MTPLUS.append(sfs4.theta_w())
    theta_w_total_MTPLUS.append(sfst.theta_w())
    sfs_silent_MTPLUS.append(sfs4.sfs)
    sfs_notSilent_MTPLUS.append(sfs0.sfs)
    sfs_t_MTPLUS.append(sfst.sfs)   
    ###################################
    ## MTMINUS
    sfst,sfs0,sfs4 = alignment2sfs_w_degen(aln, 8, degen_map, samples=mtMinus_sampleIDs, start=aln_start, end=aln_end)
    ##### record ALL results
    alleles_MTMINUS.append(sfst.alleles)
    sites_MTMINUS.append(sfst.sites())
    invariant_MTMINUS.append(sfst.invariant())
    variant_MTMINUS.append(sfst.variant())
    theta_pi_notSilent_MTMINUS.append(sfs0.theta_pi())
    theta_pi_silent_MTMINUS.append(sfs4.theta_pi())
    theta_pi_total_MTMINUS.append(sfst.theta_pi())
    theta_w_notSilent_MTMINUS.append(sfs0.theta_w())
    theta_w_silent_MTMINUS.append(sfs4.theta_w())
    theta_w_total_MTMINUS.append(sfst.theta_w())
    sfs_silent_MTMINUS.append(sfs4.sfs)
    sfs_notSilent_MTMINUS.append(sfs0.sfs)
    sfs_t_MTMINUS.append(sfst.sfs)   
    ###################################
    ## population stats
    pop_stats= population_poly(aln, 12, degen_map, mtPlus_sampleIDs, mtMinus_sampleIDs, start=aln_start, end=aln_end)
    private_MTPLUS.append(  pop_stats['private1'])
    private_MTMINUS.append( pop_stats['private2'])
    shared.append(pop_stats['shared'])
    fixed.append(pop_stats['fixed'])
    Fst.append(pop_stats['fst'])
    #######
    ## GC_silent
    GCsilent.append(len([1 for i,j in zip(special_consensus(aln)[aln_start:aln_end],degen_map[aln_start:aln_end]) if i in 'GCgc' and j =='4'])/
               len([1 for i,j in      zip(special_consensus(aln)[aln_start:aln_end],degen_map[aln_start:aln_end]) if i in 'AaTtGCgc' and j =='4']))


#########################################################
#### DF the results #####################################
#########################################################
print(GCsilent)
gametolog_results = pd.DataFrame(
    {'window' : windows,
     'start_coords' : start_coords,
     'end_coords' : end_coords,
     'alleles_ALL' : alleles_ALL, 
     'sites_ALL' : sites_ALL, 
     'invariant_ALL' : invariant_ALL, 
     'variant_ALL' : variant_ALL, 
     'theta_pi_notSilent_ALL' : theta_pi_notSilent_ALL, 
     'theta_pi_silent_ALL' : theta_pi_silent_ALL, 
     'theta_pi_total_ALL' : theta_pi_total_ALL, 
     'theta_w_notSilent_ALL' : theta_w_notSilent_ALL, 
     'theta_w_silent_ALL' : theta_w_silent_ALL, 
     'theta_w_total_ALL' : theta_w_total_ALL, 
     'sfs_silent_ALL' : sfs_silent_ALL, 
     'sfs_notSilent_ALL' : sfs_notSilent_ALL, 
     'sfs_t_ALL' : sfs_t_ALL,    
     'alleles_MTPLUS' : alleles_MTPLUS, 
     'sites_MTPLUS' : sites_MTPLUS, 
     'invariant_MTPLUS' : invariant_MTPLUS, 
     'variant_MTPLUS' : variant_MTPLUS, 
     'theta_pi_notSilent_MTPLUS' : theta_pi_notSilent_MTPLUS, 
     'theta_pi_silent_MTPLUS' : theta_pi_silent_MTPLUS, 
     'theta_pi_total_MTPLUS' : theta_pi_total_MTPLUS, 
     'theta_w_notSilent_MTPLUS' : theta_w_notSilent_MTPLUS, 
     'theta_w_silent_MTPLUS' : theta_w_silent_MTPLUS, 
     'theta_w_total_MTPLUS' : theta_w_total_MTPLUS, 
     'sfs_silent_MTPLUS' : sfs_silent_MTPLUS, 
     'sfs_notSilent_MTPLUS' : sfs_notSilent_MTPLUS, 
     'sfs_t_MTPLUS' : sfs_t_MTPLUS,         
     'alleles_MTMINUS' : alleles_MTMINUS, 
     'sites_MTMINUS' : sites_MTMINUS, 
     'invariant_MTMINUS' : invariant_MTMINUS, 
     'variant_MTMINUS' : variant_MTMINUS, 
     'theta_pi_notSilent_MTMINUS' : theta_pi_notSilent_MTMINUS, 
     'theta_pi_silent_MTMINUS' : theta_pi_silent_MTMINUS, 
     'theta_pi_total_MTMINUS' : theta_pi_total_MTMINUS, 
     'theta_w_notSilent_MTMINUS' : theta_w_notSilent_MTMINUS, 
     'theta_w_silent_MTMINUS' : theta_w_silent_MTMINUS, 
     'theta_w_total_MTMINUS' : theta_w_total_MTMINUS, 
     'sfs_silent_MTMINUS' : sfs_silent_MTMINUS, 
     'sfs_notSilent_MTMINUS' : sfs_notSilent_MTMINUS, 
     'sfs_t_MTMINUS' : sfs_t_MTMINUS, 
     'private_MTPLUS' : private_MTPLUS, 
     'private_MTMINUS' : private_MTMINUS, 
     'shared' : shared, 
     'fixed' : fixed, 
     'Fst' : Fst,
     'GCsilent' : GCsilent
    })


gametolog_results.to_csv("polymorphism_all_windowed.gametolog.csv", index=False)
gametolog_results.head()


 12%|█▏        | 77/646 [51:38<6:23:46, 40.47s/it]

ZeroDivisionError: division by zero